In [2]:
#Import Dependicies
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
#Define Resourse path
url="/content/charity_data.csv"

In [4]:
# Import and read the charity_data.csv.
charity_df = pd.read_csv(url)
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
#List the columns in the dataframe
charity_df.columns

Index(['EIN', 'NAME', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
       'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT',
       'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')

In [6]:
#Drop the EIN and NAME columns from the Dataframe
charity_df.drop(['EIN', 'NAME'],axis=1,inplace=True)

In [7]:
charity_df["STATUS"].unique()

array([1, 0])

In [8]:
charity_df["INCOME_AMT"].unique()

array(['0', '1-9999', '10000-24999', '100000-499999', '10M-50M',
       '25000-99999', '50M+', '1M-5M', '5M-10M'], dtype=object)

In [9]:
charity_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


In [10]:
#Find Number od Unique values in each Column
charity_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [11]:
#bin "rare" categorical variables together in a new value, "Other"
cutoff = 100
other_columns_arr = charity_df["CLASSIFICATION"].value_counts()[charity_df["CLASSIFICATION"].value_counts()<cutoff].keys()
other_columns_arr



Index(['C2800', 'C7100', 'C1300', 'C1280', 'C1230', 'C1400', 'C7200', 'C2300',
       'C1240', 'C8000', 'C7120', 'C1500', 'C1800', 'C6000', 'C1250', 'C8200',
       'C1238', 'C1278', 'C1235', 'C1237', 'C7210', 'C2400', 'C1720', 'C4100',
       'C1257', 'C1600', 'C1260', 'C2710', 'C0', 'C3200', 'C1234', 'C1246',
       'C1267', 'C1256', 'C2190', 'C4200', 'C2600', 'C5200', 'C1370', 'C1248',
       'C6100', 'C1820', 'C1900', 'C1236', 'C3700', 'C2570', 'C1580', 'C1245',
       'C2500', 'C1570', 'C1283', 'C2380', 'C1732', 'C1728', 'C2170', 'C4120',
       'C8210', 'C2561', 'C4500', 'C2150'],
      dtype='object')

In [12]:
charity_df["CLASSIFICATION"]

0        C1000
1        C2000
2        C3000
3        C2000
4        C1000
         ...  
34294    C1000
34295    C3000
34296    C2000
34297    C3000
34298    C1000
Name: CLASSIFICATION, Length: 34299, dtype: object

In [13]:
charity_df["CLASSIFICATION"][charity_df["CLASSIFICATION"].isin(other_columns_arr)] = "Other"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [14]:
charity_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [15]:
cutoff = 100
other_columns_arr1 = charity_df["APPLICATION_TYPE"].value_counts()[charity_df["APPLICATION_TYPE"].value_counts()<cutoff].keys()
other_columns_arr1

Index(['T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17'], dtype='object')

In [16]:
# pd.get_dummies(data, prefix=None, prefix_sep=’_’, dummy_na=False, columns=None, sparse=False, drop_first=False, dtype=None)
ohe_columns = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
       'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']
ohe_df = pd.get_dummies(charity_df, columns=ohe_columns)

In [17]:
charity_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [18]:
len(ohe_df.columns)

58

In [19]:
# Split our preprocessed data into our features and target arrays
y = ohe_df['IS_SUCCESSFUL'].values
y

# drop 'IS SUCCESSFUL'
X = ohe_df.drop('IS_SUCCESSFUL', axis=1).values
X

array([[       1,     5000,        1, ...,        0,        1,        0],
       [       1,   108590,        0, ...,        0,        1,        0],
       [       1,     5000,        0, ...,        0,        1,        0],
       ...,
       [       1,     5000,        0, ...,        0,        1,        0],
       [       1,     5000,        0, ...,        0,        1,        0],
       [       1, 36500179,        0, ...,        0,        1,        0]])

In [20]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [21]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len( X_train_scaled[0])
hidden_nodes_layer1=8
hidden_nodes_layer2=16
hidden_nodes_layer3=16
nn = tf.keras.models.Sequential()


# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 464       
                                                                 
 dense_1 (Dense)             (None, 16)                144       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 625
Trainable params: 625
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,validation_split=0.15, epochs=100)

Epoch 1/100
684/684 [==============================] - 3s 3ms/step - loss: 0.6065 - accuracy: 0.6923 - val_loss: 0.5591 - val_accuracy: 0.7287
Epoch 2/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5653 - accuracy: 0.7220 - val_loss: 0.5518 - val_accuracy: 0.7331
Epoch 3/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5596 - accuracy: 0.7262 - val_loss: 0.5481 - val_accuracy: 0.7372
Epoch 4/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5559 - accuracy: 0.7277 - val_loss: 0.5478 - val_accuracy: 0.7357
Epoch 5/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5540 - accuracy: 0.7294 - val_loss: 0.5455 - val_accuracy: 0.7346
Epoch 6/100
684/684 [==============================] - 2s 3ms/step - loss: 0.5529 - accuracy: 0.7306 - val_loss: 0.5458 - val_accuracy: 0.7336
Epoch 7/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5519 - accuracy: 0.7303 - val_loss: 0.5455 - val_accuracy: 0.7359

Adding a Third Hidden Layer, changing the activation function from Relu to Sigmoid and increasing the number of Epochs from 100 to 150  to increase the Predictive Accuracy

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len( X_train_scaled[0])
hidden_nodes_layer1=8
hidden_nodes_layer2=16
hidden_nodes_layer3=16
nn = tf.keras.models.Sequential()


# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation='relu'))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='relu'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 8)                 464       
                                                                 
 dense_8 (Dense)             (None, 16)                144       
                                                                 
 dense_9 (Dense)             (None, 16)                272       
                                                                 
 dense_10 (Dense)            (None, 1)                 17        
                                                                 
Total params: 897
Trainable params: 897
Non-trainable params: 0
_________________________________________________________________


In [29]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

# Train the model
fit_model = nn.fit(X_train_scaled,y_train,validation_split=0.20, epochs=150)

Epoch 1/150
644/644 [==============================] - 2s 3ms/step - loss: 0.5652 - accuracy: 0.7326 - val_loss: 0.5683 - val_accuracy: 0.7378
Epoch 2/150
644/644 [==============================] - 2s 2ms/step - loss: 0.5547 - accuracy: 0.7320 - val_loss: 0.5540 - val_accuracy: 0.7394
Epoch 3/150
644/644 [==============================] - 2s 2ms/step - loss: 0.5516 - accuracy: 0.7332 - val_loss: 0.5477 - val_accuracy: 0.7390
Epoch 4/150
644/644 [==============================] - 2s 2ms/step - loss: 0.5532 - accuracy: 0.7323 - val_loss: 0.5511 - val_accuracy: 0.7353
Epoch 5/150
644/644 [==============================] - 2s 2ms/step - loss: 0.5510 - accuracy: 0.7325 - val_loss: 0.5526 - val_accuracy: 0.7370
Epoch 6/150
644/644 [==============================] - 1s 2ms/step - loss: 0.5510 - accuracy: 0.7338 - val_loss: 0.5520 - val_accuracy: 0.7355
Epoch 7/150
644/644 [==============================] - 2s 3ms/step - loss: 0.5507 - accuracy: 0.7329 - val_loss: 0.5495 - val_accuracy: 0.7318

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5773 - accuracy: 0.7277 - 333ms/epoch - 1ms/step
Loss: 0.5773361325263977, Accuracy: 0.7276967763900757


In [30]:
# Export our model to HDF5 file
from google.colab import files

nn.save('/content/AlphabetSoupCharity.h5')
files.download('/content/AlphabetSoupCharity.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>